In [4]:
from typing  import Tuple
from helpers import data

In [3]:
bp_seats = data(5)
bp_seats[:5]

['FBFBFFBLLR', 'FBBBFFBLLL', 'BFBFFFFRRR', 'BFBBBFBRLL', 'FBFBBFBLLL']

**Part 1:** Find the highest seat ID

In [11]:
def seat_row_column(bp_seat) -> Tuple[int, int]: 
    """Convert binary-partitioned seat (7 characters F/B), 
    (3 characters L/R) to a tuple (row, column).
    """
    row_bp = bp_seat[:7]
    col_bp = bp_seat[-3:]
    # Row 
    row = 0 
    row_max = 128
    for i, binary in enumerate(row_bp):
        row += row_max / 2**(i+1) * (binary == "B")
    # Column 
    col = 0 
    col_max = 8 
    for i, binary in enumerate(col_bp):
        col += col_max / 2**(i+1) * (binary == "R")
    return tuple(map(int, (row, col)))

def seat_id(bp_seat) -> int: 
    row, col = seat_row_column(bp_seat)
    return row * 8 + col

In [12]:
def attempt_one():
    return max(seat_id(bp_seat) for bp_seat in bp_seats)
attempt_one()

938

In [14]:
def attempt_two():
    return max(map(seat_id, bp_seats))
attempt_two()

938

`seat_row_column` is ugly. How do I make it better?

In [24]:
def binary_partitioned_to_value(bp: str, one_char: chr = "1") -> int: 
    """Decode binary-partitioned string where one_char means 1/yes."""
    val = 0
    for i, c in enumerate(bp): 
        # I think this is clean enough...
        val += 2**(len(bp) - (i+1)) * (c == one_char)
    return val

def binary_to_row_col(bp_seat) -> Tuple[int, int]:
    """
    Convert a binary-partitioned seat (written as 7 characters F/B 
    followed by 3 characters L/R) to a tuple (row, column). 
    
    >>> binary_to_row_col("FBFBBFFRLR")
    (44, 5)
    >>> binary_to_row_col("BFFFBBFRRR")
    (70, 7)
    >>> binary_to_row_col("FFFBBBFRRR")
    (14, 7)
    >>> binary_to_row_col("BBFFBBFRLL")
    (102, 4)
    """
    binary_row = bp_seat[:7]
    binary_col = bp_seat[7:]
    return (binary_partitioned_to_value(binary_row, "B"), 
            binary_partitioned_to_value(binary_col, "R")) 

In [23]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=4)

In [25]:
def attempt_three():
    return max(seat_id(bp_seat) for bp_seat in bp_seats)
attempt_three()

938

In [26]:
# Dangit Norvig 
ID = int # Type 
def seat_id(seat: str, table=str.maketrans("FBLR", "0101")) -> ID:
    return ID(seat.translate(table), base=2)

def attempt_four():
    return max(seat_id(seat) for seat in bp_seats)
attempt_four()

938

In [28]:
from timeit import timeit 
print(timeit(attempt_three, number=1000))
print(timeit(attempt_four, number=1000))

0.4691725210000186
0.45491470899992237


**Part 2:** My seat is missing from the list. Other seats are missing too, but my seat will be the only one where the seats with ID +1 and -1 will exist. Find my seat ID. 

In [29]:
def attempt_one():
    ids = sorted([seat_id(bp_seat) for bp_seat in bp_seats])
    for i in range(len(ids) - 1): 
        if ids[i] + 1 == ids[i+1] - 1:
            return ids[i] + 1 
attempt_one()

696

In [35]:
def attempt_two():
    # They're unique so can just make a set 
    ids = set(map(seat_id, bp_seats))
    for i in ids: 
        if i + 1 not in ids and i + 2 in ids: 
            return i + 1
attempt_two()

696

In [36]:
# Dammit Norvig 
# I didn't use information that only other missing ones are at the bottom or end 
def attempt_three(): 
    ids = set(map(seat_id, bp_seats))
    [missing] = set(range(min(ids), max(ids))) - ids 
    return missing
attempt_three()

696